In [6]:
class Player:
    def __init__(self, name, kills, deaths, wins, losses):
        self.name = name
        self.kills = kills
        self.deaths = deaths
        self.wins = wins
        self.losses = losses

p1 = Player('Cat', 2, 2, 2, 2)
p1.kills += 11
p1.kills

TypeError: __init__() missing 2 required positional arguments: 'wins' and 'losses'